In this script, I construct the target convolutional neural network and train it on the MNIST dataset. I decided for an architecture with large kernels, loss stride and high padding. The reason for this choice is that the activation maps before the second covolutional layer are comparatavely bigger. This allows for an easier study of interpretability in Experiment 11.

The architecture is as follows:

1. **Conv1**: 1→8 channels, 9x9 kernel, stride 1, padding 4, ReLU, MaxPool (2x2), He Initialization.
2. **Conv2**: 8→16 channels, 9x9 kernel, stride 1, padding 4, ReLU, MaxPool (2x2), He Initialization.
3. **FC Layer**: 16 * 7 * 7 input, 10 output (MNIST classes), He Initialization with ReLU.


In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # Initialize convolutional layer 1 with He Initialization
        # Increased kernel size to 9 and number of filters remains 8
        self.conv1 = self._init_conv_layer(1, 8, 9, 1, 4, 'he')  # Padding adjusted to 4

        # Initialize convolutional layer 2 with He Initialization
        # Increased kernel size to 9 and number of filters remains 16
        self.conv2 = self._init_conv_layer(8, 16, 9, 1, 4, 'he')  # Padding adjusted to 4

        # Initialize fully connected output layer
        # Input dimension remains the same
        self.out = nn.Linear(16 * 7 * 7, 10)
        nn.init.kaiming_normal_(self.out.weight, nonlinearity='relu')

    def _init_conv_layer(self, in_channels, out_channels, kernel_size, stride, padding, init_method):
        """Initializes a convolutional layer followed by a ReLU activation and MaxPool."""
        layer = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        if init_method == 'he':
            nn.init.kaiming_normal_(layer[0].weight, nonlinearity='relu')

        return layer

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)

        # Flatten the output for the fully connected layer
        x = x.view(x.size(0), -1)

        output = self.out(x)
        return output, x


Now we instantiate the neural network, train it on the MNIST data set and subsequently evaluate it.

In [ ]:
import torch.optim as optim
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader


model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data Preprocessing and Loading
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

# Training Loop
num_epochs = 10

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs, _ = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}] completed.')

# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs, _ = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f'Test Accuracy of the model: {test_accuracy}%')

100%|██████████| 9912422/9912422 [00:00<00:00, 245956277.29it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 103446365.35it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 18130056.62it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 22229321.78it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/10] completed.
Epoch [2/10] completed.
Epoch [3/10] completed.
Epoch [4/10] completed.
Epoch [5/10] completed.
Epoch [6/10] completed.
Epoch [7/10] completed.
Epoch [8/10] completed.
Epoch [9/10] completed.
Epoch [10/10] completed.
Test Accuracy of the model: 98.85%


Lastly, we save the weights so that we can reuse the target CNN for later analysis.

In [ ]:
torch.save(model.state_dict(), "mnistTwoLayersFilterSize9And16Filters.pth")
from google.colab import files
files.download('mnistTwoLayersFilterSize9And16Filters.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>